# Перегрузка операторов

На лекции мы рассмотрели один из принципов объектно-ориентированного программирования - **полиморфизм**. Один из вариантов реализации полиморфизма для пользовательских объектов - это перегрузка операторов и встроенных функций. В этом практическом занятии мы с вами рассмотрим примеры перегрузки, реализуя пользовательский класс `Square`.  

## repr и str

Давайте реализуем класс `Square`, который будет позволять описывать квадраты в наших программах. У класса `Square` будет один служебный атрибут - `_side` - длина стороны квадрата. Также у этого класса будет конструктор, на вход которому передается целое число - длина стороны квадрата. Переданное целое число будет сохраняться в служебный атрибут `_side`.

Реализация может выглядеть так: 

In [ ]:
class Square:
    _side: float

    def __init__(self, side: float) -> None:
        if side < 0:
            raise ValueError(
                "side must be greater than or equal to 0"
                f" but the next value was given: {side}"
            )

        self._side = side

Как мы знаем из предыдущего занятия, по умолчанию экземпляры пользовательских классов в Python не имеют информативного строкового представления. Т.е. если мы попытаемся получить строковое представления экземпляра класса `Square`, мы получим абсолютно неинформативную строку:

In [ ]:
square = Square(side=5)
print(
    f"repr: {square!r}",
    f"str: {square!s}",
    sep="\n",
)

Мы знаем, что определить вменяемое строковое представления объекта можно с помощью декоратора `dataclass`. Однако такой подход определения строковых представлений удобен далеко не всегда. Определение строкового представления объекта с помощью декоратора `dataclass` сильно ограничено. Как у разработчиков, у нас нет полного контроля за тем, как это представление будет выглядеть в итоге. С помощью `dataclass` мы не можем настроить представление под наши нужды.

Однако мы можем пойти по другому пути и научить встроенные объекты `repr()` и `str` работать с нашим объектом по-особенному. Для этого нам необходимо определить специальные методы `__repr__` и  `__str__` для нашего объекта `Square`. Каждый из этих методов не должен принимать на вход ни одного аргумента, кроме `self`, а результатом выполнения этих методов должны быть объекты строковых типов данных.

In [ ]:
class Square:
    _side: float

    def __init__(self, side: float) -> None:
        if side < 0:
            raise ValueError(
                "side must be greater than or equal to 0"
                f" but the next value was given: {side}"
            )

        self._side = side

    def __str__(self) -> str:
        print("call str")
        return f"square with side '{self._side}'"

    def __repr__(self) -> str:
        print("call repr")
        return f"Square(side={self._side})"

In [ ]:
square = Square(side=5)
print(
    f"repr: {square!r}",
    f"str: {square!s}",
    sep="\n",
)

Как видно из этого примера, теперь, при вызове встроенных объектов `repr()` и `str` с объектом типа `Square` в качестве аргумента, для получения строкового представления объекта вызываются соответствующие магические методы. Важно понять, что явно эти методы вызывать не нужно, хотя это и корректно с точки зрения языка. Однако подобный стиль является нежелательным, и другие разработчики могут вас не понять. Вызов магических методов происходит неявно внутри объектов `repr()` и `str`.

Если `repr()` и `str` должны возвращать одинаковые строковые представления, вы можете определить только `__repr__`. В этом случае отсутствующий специальный метод `__str__` будет использовать определенную реализацию специального метода `__repr__` для получения строкового представления:

In [ ]:
class Square:
    _side: float

    def __init__(self, side: float) -> None:
        if side < 0:
            raise ValueError(
                "side must be greater than or equal to 0"
                f" but the next value was given: {side}"
            )

        self._side = side

    def __repr__(self) -> str:
        return f"square with side '{self._side}'"

In [ ]:
square = Square(side=5)
print(
    f"repr: {square!r}",
    f"str: {square!s}",
    sep="\n",
)

## Логическое представление

Не лишним бывает определить логическое представление для объекта. Пусть наш объект типа `Square` имеет логическое представление `True`, если его площадь больше нуля, и `False` - иначе. Для сравнения числа с плавающей точкой с нулем будем использовать метод `isclose()` из модуля стандартной библиотеки `math`. 

In [ ]:
import math


class Square:
    _side: float

    def __init__(self, side: float) -> None:
        if side < 0:
            raise ValueError(
                "side must be greater than or equal to 0"
                f" but the next value was given: {side}"
            )

        self._side = side

    def __repr__(self) -> str:
        return f"square with side '{self._side}'"
    
    def __bool__(self) -> bool:
        return not math.isclose(self.get_area(), 0)

    def get_area(self) -> float:
        return self._side ** 2

In [ ]:
print(
    bool(Square(side=5)),
    bool(Square(side=0)),
    sep="\n",
)

Чтобы иметь возможность получать логическое представление нашего объекта типа `Square`, мы научили встроенный объект `bool` работать с нашим объектом. Для этого мы определили магический метод `__bool__` в нашем объекте-класса `Square`. Этот метод принимает на вход экземпляр данного класса и возвращает объект логического типа данных. В теле данного метода мы вычисляем площадь квадрата с помощью метода `get_area()` и возвращаем результат сравнения площади с 0.

## Операции сравнения

Перегрузим операторы `==` и `!=` для объектов типа `Square`. Будем считать, что два квадрата равны тогда и только тогда, когда равны длины их сторон. Для сравнения так же будем использовать метод `isclose()` из модуля `math`, т.к. стороны квадратов описываются числами с плавающей точкой.

Для перегрузки операторов `==` и `!=` необходимо реализовать специальные методы `__eq__` (сокращение от *equal*) и `__ne__` (сокращение от *not equal*). На вход оба этих метода принимают текущий экземпляр класса и объект, с которым происходит сравнение. В нашем случае это объект типа `Square`. В качестве результата эти методы традиционно возвращают объекты логического типа данных. Но если логика вашей программы требует иного, вы можете вернуть из этих методов объект нужного вам типа данных. Однако делать это крайне не рекомендуется.

In [ ]:
import math


class Square:
    _side: float

    def __init__(self, side: float) -> None:
        if side < 0:
            raise ValueError(
                "side must be greater than or equal to 0"
                f" but the next value was given: {side}"
            )

        self._side = side

    def __repr__(self) -> str:
        return f"square with side '{self._side}'"
    
    def __bool__(self) -> bool:
        return not math.isclose(self.get_area(), 0)
    
    def __eq__(self, other: Square) -> bool:
        return math.isclose(self._side, other._side)
    
    def __ne__(self, other: Square) -> bool:
        return not (self == other)

    def get_area(self) -> float:
        return self._side ** 2

In [ ]:
square1 = Square(5)
square2 = Square(3.14)

print(
    square1 == square2,
    square1 != square2,
    sep="\n",
)

Обратите внимание на реализацию метода `__ne__`. Метод `__ne__` был реализован, как отрицание результата выполнения метода `__eq__`. На самом деле такое определение излишне. Мы можем реализовать только метод `__eq__`, а Python по умолчанию сам будет использовать эту реализацию для определения метода `__ne__`.

In [ ]:
import math


class Square:
    _side: float

    def __init__(self, side: float) -> None:
        if side < 0:
            raise ValueError(
                "side must be greater than or equal to 0"
                f" but the next value was given: {side}"
            )

        self._side = side

    def __repr__(self) -> str:
        return f"square with side '{self._side}'"
    
    def __bool__(self) -> bool:
        return not math.isclose(self.get_area(), 0)
    
    def __eq__(self, other: Square) -> bool:
        print("call __eq__")
        return math.isclose(self._side, other._side)

    def get_area(self) -> float:
        return self._side ** 2

In [ ]:
square1 = Square(5)
square2 = Square(3.14)

print(
    square1 == square2,
    square1 != square2,
    sep="\n",
)

Также обратите внимание, что эти методы вызываются интерпретатором неявно при обработке соответствующих операций. В самом коде явно вызывать эти методы не нужно. Вместо этого вы можете использовать соответствующие операторы напрямую.

## Отношения порядка

Определим для объектов типа `Square` отношение порядка. Будем считать, что один квадрат больше другого, в том и только в том случае, когда его площадь больше.

Для реализации отношения строгого порядка мы должны выполнить перегрузку операторов `>` и `<`. Для этого нам необходимо определить специальные методы `__gt__` (сокращение от *greater than*) и `__lt__` (сокращение от *lower than*). Оба метода на вход принимают два аргумента по аналогии с `__eq__` и `__ne__`.

In [ ]:
import math


class Square:
    _side: float

    def __init__(self, side: float) -> None:
        if side < 0:
            raise ValueError(
                "side must be greater than or equal to 0"
                f" but the next value was given: {side}"
            )

        self._side = side

    def __repr__(self) -> str:
        return f"square with side '{self._side}'"
    
    def __bool__(self) -> bool:
        return not math.isclose(self.get_area(), 0)
    
    def __eq__(self, other: Square) -> bool:
        return math.isclose(self._side, other._side)
    
    def __gt__(self, other: Square) -> bool:
        return self != other and self._side > other._side

    def __lt__(self, other: Square) -> bool:
        return self != other and self._side < other._side

    def get_area(self) -> float:
        return self._side ** 2

In [ ]:
square1 = Square(5)
square2 = Square(3.14)

print(
    square1 > square2,
    square1 < square2,
    sep="\n",
)

Для определения отношения нестрогого порядка мы можем перегрузить операторы `>=` и `<=`. Делается это путем определения специальных методов `__ge__` (сокращение от *greater or equal*) и `__le__` (сокращение от *lower or equal*). Сигнатуры этих методов похожи на сигнатуры методов для определения отношения строгого порядка. Реализация этих методов предлагается вам в качестве самостоятельного упражнения.

## Аддитивные операции

### Прямые операции

Определим операцию сложения и вычитания для наших объектов типа `Square`. Результатом сложения квадратов будет новый объект типа `Square`, площадь которого равна сумме площадей исходных квадратов. Отсюда мы сможем вычислить длину стороны нового квадрата. Аналогично определим вычитание квадратов. Результатом выполнения обеих операций должен быть новый объект типа `Square`.

Для перегрузки бинарных операторов `+` и `-` мы должны определить специальные методы `__add__` и `__sub__`, соответственно:

In [ ]:
import math


class Square:
    _side: float

    def __init__(self, side: float) -> None:
        self._side = side

    def __repr__(self) -> str:
        return f"square with side '{self._side}'"
    
    def __bool__(self) -> bool:
        return not math.isclose(self.get_area(), 0)
    
    def __eq__(self, other: Square) -> bool:
        return math.isclose(self._side, other._side)
    
    def __gt__(self, other: Square) -> bool:
        return self != other and self._side > other._side

    def __lt__(self, other: Square) -> bool:
        return self != other and self._side < other._side
    
    def __add__(self, other: Square) -> Square:
        area_common = self.get_area() + other.get_area()
        return Square(side=area_common ** 0.5)

    def __sub__(self, other: Square) -> Square:
        if self < other:
            raise ValueError(
                f"impossible to subtract {other} from {self}"
            )
        
        area_common = self.get_area() - other.get_area()
        return Square(side=area_common ** 0.5)

    def get_area(self) -> float:
        return self._side ** 2

In [ ]:
square1 = Square(3)
square2 = Square(4)

print(
    square1 + square2,
    square2 - square1,
    sep="\n",
)


try:
    square1 - square2

except ValueError as exc:
    print(exc)

Обратите внимание на реализацию метода `__sub__`. Перед началом вычитания мы проверяем, что вычитание происходит из квадрата с большей площадью, чтобы не получить отрицательную площадь.

Помимо сложения и вычитание квадратов, давайте определим операцию сложения и вычитания с числами. При алгебраическом суммировании объекта типа `Square` с объектом числового типа данных будем считать, что объект числового типа данных представляет квадрат, длина стороны которого задана значением объекта числового типа данных. Для определения этих операций дополним определения методов `__add__` и `__sub__`:

In [ ]:
import math
from numbers import Real


class Square:
    _side: float

    def __init__(self, side: float) -> None:
        self._side = side

    def __repr__(self) -> str:
        return f"square with side '{self._side}'"
    
    def __bool__(self) -> bool:
        return not math.isclose(self.get_area(), 0)
    
    def __eq__(self, other: Square) -> bool:
        return math.isclose(self._side, other._side)
    
    def __gt__(self, other: Square) -> bool:
        return self != other and self._side > other._side

    def __lt__(self, other: Square) -> bool:
        return self != other and self._side < other._side
    
    def __add__(self, other: Square | Real) -> Square:
        if isinstance(other, Real):
            other = Square(side=other)

        area_common = self.get_area() + other.get_area()
        return Square(side=area_common ** 0.5)

    def __sub__(self, other: Square | Real) -> Square:
        if isinstance(other, Real):
            other = Square(side=other)

        if self < other:
            raise ValueError(
                f"impossible to subtract {other} from {self}"
            )
        
        area_common = self.get_area() - other.get_area()
        return Square(side=area_common ** 0.5)

    def get_area(self) -> float:
        return self._side ** 2

In [ ]:
square1 = Square(3)
square2 = Square(4)

print(
    square1 + square2,
    square2 - square1,
    square1 + 4,
    square2 - 3,
    sep="\n",
)


try:
    square1 - square2

except ValueError as exc:
    print(exc)

Реализации методов `__add__` и `__sub__` для вычисления алгебраических сумм могли бы быть эффективнее. Однако мы намеренно упростили реализацию, чтобы вам было легче читать этот код.

Также обратите внимание на использование объекта `Real`. Опуская подробности, можно сказать, что этот объект был использован для корректной проверки типов с помощью `isinstance`. Дело в том, что объект числового типа данных может быть как целым числом, так и числом с плавающей точкой. Объект `Real` позволяет без использования сложных конструкций, типа `isinstance(obj, int | float)`, проверить, что объект является или целым числом, или числом с плавающей точкой.

### Отраженные операции

Текущая реализация объекта `Square` имеет существенный недостаток. Операция сложения объекта типа `Square` с  объектом числового типа не коммутативна:

In [ ]:
try:
    1 + Square(5)

except TypeError as exc:
    print(exc)

Это происходит потому, что при сложении объектов с помощью бинарного оператора `+`, интерпретатор действует так:
- Если у левого операнда определен специальный метод `__add__`, интерпретатор вызывает его. Иначе интерпретатор переходит к анализу правого операнда.
- Если был вызван метод `__add__` левого операнда и в результате его выполнения был получен осмысленный результат, выполнение прекращается. 
- Если был вызван метод `__add__` левого операнда и в результате его выполнения был получен объект-синглтон `NotImplemented`, интерпретатор переходит к анализу правого операнда.
- При анализе правого операнда интерпретатор проверяет наличие у него специального метода `__radd__`. Если этот метод не определен, результат выполнения операции - исключение типа `TypeError`.
- Если `__radd__` определен и результат его выполнения - `NotImplemented`, интерпретатор возбудит исключение.
- Если `__radd__` определен и результат его выполнения - не `NotImplemented`, интерпретатор вернет полученный объект в качестве результата выполнения бинарной операции.

Проиллюстрируем эту логику работы бинарной операции `+` схемой:

![scheme](./images/scheme.png)

Логика работы прочих бинарных арифметических операций аналогична.

Т.е. для того, чтобы операция сложения была коммутативной при работе с объектами числовых типов данных и объектами типа `Square`, мы должны реализовать специальный метод `__radd__`. `r` в имение метода указывает на то, что метод вызывается у операнда, расположенного справа от оператора.

In [ ]:
import math
from numbers import Real


class Square:
    _side: float

    def __init__(self, side: float) -> None:
        self._side = side

    def __repr__(self) -> str:
        return f"square with side '{self._side}'"
    
    def __bool__(self) -> bool:
        return not math.isclose(self.get_area(), 0)
    
    def __eq__(self, other: Square) -> bool:
        return math.isclose(self._side, other._side)
    
    def __gt__(self, other: Square) -> bool:
        return self != other and self._side > other._side

    def __lt__(self, other: Square) -> bool:
        return self != other and self._side < other._side
    
    def __add__(self, other: Square | Real) -> Square:
        if not isinstance(other, Square | Real):
            return NotImplemented

        if isinstance(other, Real):
            other = Square(side=other)

        area_common = self.get_area() + other.get_area()
        return Square(side=area_common ** 0.5)
    
    def __radd__(self, other: Real) -> Square:
        return self + other

    def __sub__(self, other: Square | Real) -> Square:
        if not isinstance(other, Square | Real):
            return NotImplemented

        if isinstance(other, Real):
            other = Square(side=other)

        if self < other:
            raise ValueError(
                f"impossible to subtract {other} from {self}"
            )
        
        area_common = self.get_area() - other.get_area()
        return Square(side=area_common ** 0.5)

    def get_area(self) -> float:
        return self._side ** 2

In [ ]:
print(
    Square(3) + 4,
    4 + Square(3),
    sep="\n",
)

Обратите внимание, что мы также добавили логику работы с объектом `NotImplemented` в методы, реализующие прямые аддитивные операции. Это сделано для того, чтобы другие пользовательские объекты, в случае необходимости, могли реализовать свою логику работы с нашим классом при выполнении арифметических операций.

## Мультипликативные операции

Определим операцию умножения и деления объекта типа `Square` на объект числового типа данных. Умножение квадрата на число эквивалентно созданию нового квадрата, площадь которого равна площади исходного квадрата, увеличенной в указанное число раз. Деление квадрата на число эквивалентно созданию нового квадрата, площадь которого равна площади исходного квадрата, уменьшенной в указанное число раз.

Для перегрузки операторов `*` и `/` необходимо определить методы `__mul__` и `__truediv__`:

In [ ]:
import math
from numbers import Real


class Square:
    _side: float

    def __init__(self, side: float) -> None:
        self._side = side

    def __repr__(self) -> str:
        return f"square with side '{self._side}'"
    
    def __bool__(self) -> bool:
        return not math.isclose(self.get_area(), 0)
    
    def __eq__(self, other: Square) -> bool:
        return math.isclose(self._side, other._side)
    
    def __gt__(self, other: Square) -> bool:
        return self != other and self._side > other._side

    def __lt__(self, other: Square) -> bool:
        return self != other and self._side < other._side
    
    def __add__(self, other: Square | Real) -> Square:
        if not isinstance(other, Square | Real):
            return NotImplemented

        if isinstance(other, Real):
            other = Square(side=other)

        area_common = self.get_area() + other.get_area()
        return Square(side=area_common ** 0.5)
    
    def __radd__(self, other: Real) -> Square:
        return self + other

    def __sub__(self, other: Square | Real) -> Square:
        if not isinstance(other, Square | Real):
            return NotImplemented

        if isinstance(other, Real):
            other = Square(side=other)

        if self < other:
            raise ValueError(
                f"impossible to subtract {other} from {self}"
            )
        
        area_common = self.get_area() - other.get_area()
        return Square(side=area_common ** 0.5)

    def __mul__(self, scale: Real) -> Square:
        if not isinstance(scale, Real):
            return NotImplemented

        area = self.get_area() * scale
        return Square(side=area ** 0.5)

    def __truediv__(self, scale: Real) -> Square:
        if not isinstance(scale, Real):
            return NotImplemented

        return self * (1 / scale)

    def get_area(self) -> float:
        return self._side ** 2

In [ ]:
square = Square(2)

print(
    square * 4,
    square / 4,
    sep="\n",
)

Методы `__mul__` и `__truediv__` являются прямыми. Они также имеют отраженные аналоги `__rmul__` и `__rtruediv__`, соответственно. Реализация отраженного умножения предлагается вам в качестве самостоятельного упражнения.

## Материалы

В данном занятии были рассмотрены далеко не все специальные методы для перегрузки операторов и встроенных функций. Для детального ознакомления с полным списком специальных методов вы можете воспользоваться [официальной документацией](https://docs.python.org/3/library/operator.html).

## Задание

По [данной ссылке](https://github.com/EvgrafovMichail/python_mipt_dafe_tasks/blob/main/conditions/lesson11/task.md) вы найдете самостоятельное задание. Задание необходимо выполнить до 23:59 7 декабря.